## 📝 Free/Freemium API Provider Comparison
### Provider Research Log
| Provider      | Data Completeness | Data Depth     | Data Freshness   | Rate Limits          | Python SDK                       | Documentation Quality & Community Support             | Notes               |
|:-------------:|:-----------------:|---------------:|:----------------:|:--------------------:|:--------------------------------:|:-----------------------------------------------------:|:-------------------:|
| Alpha Vantage | ✅               | 20+ years      | Up-to-date       | 25 requests/day      | Easy -- `REST API`, `requests`, `json` | [Comprehensive](https://www.alphavantage.co/documentation/) | ...      |
| Tiingo | ✅ | 30+ years | Up-to-date | 1000 requests/day, 50 requests/hour; 500 symbols/month | Easy -- `REST API`, `requests`, `json`| [Comprehensive](https://www.tiingo.com/documentation/general/overview) | ... |
| IEX Cloud/iexfinance * | N/A         | N/A           | N/A              | N/A                 | N/A                              | N/A                                                   | ...            |
| Twelve Data   | ...                | ...            | ...                | ...                   | ...                          | ...                                                   | ...                 |
| Google Finance | ...               | ...            | ...                | ...                   | ...                          | ...                                                   | ...                 |
| Polygon.io    | ...                | ...            | ...                | ...                   | ...                          | ...                                                   | ...                 |
| Finage        | ...                | ...            | ...                | ...                   | ...                          | ...                                                   | ...                 |
| Finazon       | ...                | ...            | ...                | ...                   | ...                          | ...                                                   | ...                 |
| Finnhub       | ...                | ...            | ...                | ...                   | ...                          | ...                                                   | ...                 |
| Alpaca        | ...                | ...            | ...                | ...                   | ...                          | ...                                                   | ...                 |

### Provider Pros & Cons
| Provider            |  Pros                                                                                      | Cons                                                                                       |
|:-------------------:|:------------------------------------------------------------------------------------------:|:------------------------------------------------------------------------------------------:|
| Alpha Vantage       | Awesome all-around -- Thorough Data; Great documentation; Simple Python SDK; NASDAQ licensed | Restrained service at free plan -- Low rate limits; No realtime US market data           |
| Tiingo              | Awesome all-around -- Thorough Data; Great documentation; Simple Python SDK; Widely adopted | Restrained service at free plan -- Relatively low rate limits; Fewscreeners               |
| IEX Cloud/iexfinance * | N/A                                                                                    | N/A                                                                                       |
| Twelve Data         | ...                                                                                       | ...                                                                                       |
| Google Finance      | ...                                                                                       | ...                                                                                       |
| Polygon.io          | ...                                                                                       | ...                                                                                       |
| Finage              | ...                                                                                       | ...                                                                                       |
| Finazon             | ...                                                                                       | ...                                                                                       |
| Finnhub             | ...                                                                                       | ...                                                                                       |
| Alpaca              | ...                                                                                       | ...                                                                                       |

---
*: IEX Cloud/iexfinance has [retired their API products since August 2024](https://iexcloud.org/).

## ✅ API Shortlist
Based on the evaluation above:\
**....** is the most suitable candidate for production use because
- a.
- b.

**....** may serve as a secondary backup because
- a.
- b.

**....** is also useful because
- a.
- b.

Data Fields: OHLCV + adjusted close?
Data Coverage: How far back does the historical data go? up-to-date?
Rate Limits: requests/minute/day.
Ease of Use: Python SDK availability, ease of authentication.
Docs & Support: Quality of documentation and community examples.
Reliability: Are there known issues like downtime or staleness?

## 📈 Visualizations

In [ ]:
# Import and Setup
import pandas as pd
import matplotlib.pyplot as plt  # or plotly
from main import fetch_daily_data

In [ ]:
# Try fetching from each API
df = fetch_daily_data("AAPL", "2023-01-01", "2023-12-31", source="tiingo")
df.head()

In [ ]:
# Plottings
# - Show OHLC candlestick for a ticker.
# - Plot volume alongside price.
df.set_index("date")[["close"]].plot(title="AAPL Daily Close")

## 🔍 Bonus: Intraday (1-minute) Data Exploration
Check each shortlisted provider for:
- (Free) Access to 1-minute historical data (at least for the past 5 trading days). How to access?
- Whether data is real-time or delayed.
- Whether intraday feeds data requires additional setup (e.g. paid plan/elevated permissions) or rate-limit considerations.
Code snippets (optional)